In [1]:
!pip install pymysql
!pip install pymssql

import pandas as pd
import numpy as np
import LibData
import LibUtils
import LibML
import LibFile
import LibEE


     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 1.3MB 12.7MB/s 


In [4]:
############# 임베딩 단계별 효과

jobName = '201216_Train_split1'

# 단계1. 원핫인코딩
x, y = LibData.GetXY_o(jobName)
x = LibData.GetMoneyness(x)
x = LibData.GetAttainedAge(x)
x = LibData.GetIrReduction(x)
scaler, x = LibUtils.ScaleStandard(x)
print('step1')
print(LibML.GetScoresCV(x,y, LibML.RandomForestDefault(len(x.columns)), 5))
print(LibML.GetScoresCV(x,y, LibML.XGBoostDefault(), 5))
# [149.02, 0.1782, 0.9267] [137.27, 0.1641, 0.9571]

# 단계2. 상품코드 컬럼 삭제
x, y = LibData.GetXYBase(jobName)
x = LibData.GetMoneyness(x)
x = LibData.GetAttainedAge(x)
x = LibData.GetIrReduction(x)
del x['ProdCode']
scaler, x = LibUtils.ScaleStandard(x)
print('step2')
print(LibML.GetScoresCV(x,y, LibML.RandomForestDefault(len(x.columns)), 5))
print(LibML.GetScoresCV(x,y, LibML.XGBoostDefault(), 5))
# [270.77, 0.3237, 0.8546] [259.69, 0.3105, 0.8368] 


# 단계3. 상품코드 EE 차원3
x, y = LibData.GetXYBase(jobName)
x = LibData.GetMoneyness(x)
x = LibData.GetAttainedAge(x)
x = LibData.GetIrReduction(x)
x, emb = LibEE.GetXEmbApplied(3, x, y, ['ProdCode'])
scaler, x = LibUtils.ScaleStandard(x)
print('step5')
print(LibML.GetScoresCV(x,y, LibML.RandomForestDefault(len(x.columns)), 5))
print(LibML.GetScoresCV(x,y, LibML.XGBoostDefault(), 5))
# [123.72, 0.148, 0.9489] # [114.57, 0.137, 0.9717]


# 단계4. 상품코드 EE 차원5
x, y = LibData.GetXYBase(jobName)
x = LibData.GetMoneyness(x)
x = LibData.GetAttainedAge(x)
x = LibData.GetIrReduction(x)
x, emb = LibEE.GetXEmbApplied(5, x, y, ['ProdCode'])
scaler, x = LibUtils.ScaleStandard(x)
print('step4')
print(LibML.GetScoresCV(x,y, LibML.RandomForestDefault(len(x.columns)), 5))
print(LibML.GetScoresCV(x,y, LibML.XGBoostDefault(), 5))
# [123.64, 0.1478, 0.9495] [114.61, 0.137, 0.9707]





step1
start learning 1 2021-01-16 13:52... end learning 2021-01-16 13:55
start learning 2 2021-01-16 13:55... end learning 2021-01-16 13:59
start learning 3 2021-01-16 13:59... end learning 2021-01-16 14:02
start learning 4 2021-01-16 14:02... end learning 2021-01-16 14:05
start learning 5 2021-01-16 14:05... end learning 2021-01-16 14:09
[149.02, 0.1782, 0.9267]
step2
start learning 1 2021-01-16 14:09... end learning 2021-01-16 14:12
start learning 2 2021-01-16 14:12... end learning 2021-01-16 14:15
start learning 3 2021-01-16 14:15... end learning 2021-01-16 14:19
start learning 4 2021-01-16 14:19... end learning 2021-01-16 14:22
start learning 5 2021-01-16 14:22... end learning 2021-01-16 14:25
[270.77, 0.3237, 0.8546]
start learning 1 2021-01-16 14:25... end learning 2021-01-16 14:28
start learning 2 2021-01-16 14:28... end learning 2021-01-16 14:30
start learning 3 2021-01-16 14:30... end learning 2021-01-16 14:33
start learning 4 2021-01-16 14:33... end learning 2021-01-16 14:36


In [ ]:
# 특정컬럼 디지타이징하여 x 변경
def GetDigitized(dt, varName, binCnt):
    startPoint = dt[varName].min() # 최소값
    endPoint = dt[varName].max() # 최대값
    bins = np.round(np.linspace(startPoint, endPoint, binCnt))
    dt[varName] = np.digitize(dt[varName], bins)

    # ★★★ Entity 값들은 0부터 시작해야함
    dt[varName] = np.array(dt[varName] - dt[varName].unique().min())
    return dt

# 범주형 변수를 수치형 변수로 바꿈
def GetNumerical(dt, varName):
    rowNames = np.sort(dt[varName].unique())
    numbers = np.arange(len(rowNames))
    mapping = pd.Series(numbers, index=rowNames)
    dt[varName] = dt.apply(lambda row: mapping[row[varName]], axis=1)
    return dt

# 특정컬럼을 0에서부터 시작하도록 바꿈
def GetZeroStartX(dt, varName):
    minVal = dt[varName].unique().min()
    dt[varName] = np.array(dt[varName] - minVal)
    return dt

# 개수만큼 전환하여 입력
def GetCardinailty(dt, varName):
    return len(dt[varName].unique())

from tensorflow.keras.layers import Input, Embedding, Dense, Reshape, Concatenate, Flatten, Dropout
from tensorflow.keras.models import Model

def GetEmbeddingLayer(input_layer, info):
    name = info[0]; cardi = info[1]; dim = info[2]
    emb_layer = Embedding(cardi, dim, name='emb_' + name)(input_layer)
    emb_layer = Flatten()(emb_layer)
    return emb_layer

# 임베딩학습을 위한 신경망모델 생성, 반환
def GetLayers(emb_info):
    input_layers = []
    emb_layers = []

    # input_layer, emb_layer 만들기
    for i in range(len(emb_info)):
        input_layer = Input(shape=(1,))
        emb_layer = GetEmbeddingLayer(input_layer, emb_info[i])
        input_layers.append(input_layer)
        emb_layers.append(emb_layer)

    # output_layer 만들기
    if len(emb_info)==1:
        output_layer = Dense(32, activation='relu')(emb_layers[0])
    else:
        output_layer = Concatenate()(emb_layers)
        output_layer = Dense(32, activation='relu')(output_layer)

    output_layer = Dense(16, activation='relu')(output_layer)
    output_layer = Dense(1)(output_layer)

    model = Model(inputs=input_layers, outputs=output_layer)
    model.compile(optimizer = 'adam', loss='mse')

    #model.summary()

    return model

# 임베딩학습을 위한 신경망모델을 학습, 반환
def GetFitted(x, emb_info, model):
    from tensorflow.keras.callbacks import EarlyStopping
    import numpy as np

    early_stop = EarlyStopping(monitor='loss', patience=10)

    x_emb = [np.array(x[info[0]]) for i, info in enumerate(emb_info)]
    y_emb = [np.array(y['pv_claim'])]

    model.fit(x=x_emb, y=y_emb, epochs=100, callbacks=[early_stop], verbose=0)

    return model

# 임베딩 매핑테이블 반환
def GetEmbTable(md, info):
    colName = info[0]; cardi = info[1]; dim = info[2]
    
    keys = pd.DataFrame(range(0, cardi), columns = [colName])
    
    colNames_New = ['ee_'+colName+'_'+ str(i).zfill(2) for i in np.arange(0, dim)]    
    w = md.get_layer('emb_' + colName).get_weights()[0]
    w = pd.DataFrame(w, columns=colNames_New)
    
    mapping = pd.concat([keys, w], axis=1)
    return mapping

# 임베딩 매핑 테이블을 적용(기존컬럼 삭제)한 x 반환
def GetXofEmb(dtFrom, colName, emb_table):    
    dt = pd.merge(dtFrom, emb_table, on=colName, how='left')
    dt = dt.drop(columns=[colName])
    return dt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# 학습하여 스코어 반환
def Learn(x,y):
    l_x, l_y = LibData.GetSplited(x,y,5)
    model = RandomForestRegressor(n_estimators =700, max_features=26, min_samples_leaf=2)
    #xTrain, xVal, yTrain, yVal = train_test_split(x, y, test_size=0.2)
    #model.fit(xTrain, yTrain['pv_claim']) 
    #y_pred = pd.Series(model.predict(xVal), index=xVal.index)
    #return LibUtils.GetScores(yVal['pv_claim'], y_pred)
    preds = LibML.GetPredCV(l_x, l_y, model, 'pv_claim')
    return LibML.GetMaesCV(l_y, preds, 'pv_claim')
  
def GetXEmbApplied(maxDim, x, vars):

     # [컬럼이름, 카테고리수, 임베딩차원] 으로 인포테이블 만듬
    emb_info = [(col, GetCardinailty(x, col), min(maxDim, (GetCardinailty(x, col)+1)//2)) for col in vars]

    # 임베딩학습을 위한 레이어만들기
    model = GetLayers(emb_info)

    # 임베딩 학습
    model = GetFitted(x, emb_info, model)

    # 임베딩 적용된 x 만들기
    for i, info in enumerate(emb_info):
        emb = GetEmbTable(model, info)
        x = GetXofEmb(x, info[0], emb)

    return x


def Execute(maxDim, x, vars, tag):

    x = GetXEmbApplied(maxDim, x, vars, tag)
    
    maes = Learn(x,y)
    summary = [str(sum(maes)/5), maes]
    
    LibUtils.WriteNote('emb_rf700', tag, summary)
    print('emb_rf700 ' + tag, summary)

In [ ]:
##### base
x, y = LibData.GetXY_o(jobName)
maes = Learn(x,y)
print('0_base', [str(sum(maes)/5), maes])

start learning 1 2020-12-27 14:03... end learning 2020-12-27 14:11
start learning 2 2020-12-27 14:11... end learning 2020-12-27 14:19
start learning 3 2020-12-27 14:19... end learning 2020-12-27 14:27
start learning 4 2020-12-27 14:27... end learning 2020-12-27 14:35
start learning 5 2020-12-27 14:35... end learning 2020-12-27 14:43
0_base ['156.25646082673944', [156.14721747803938, 157.49836106032305, 161.50239034585235, 153.77280318824512, 152.36153206123726]]


In [ ]:
maxDim = 5
tag = 'max5'

x = xBase.copy(True)
vars = ['ProdCode']
Execute(maxDim, x, vars, tag+'_1_prodCode')

x = xBase.copy(True)
vars = ['ProdCode', 'ElapsedMth']
# 경과월 --> 0에서부터 1씩 늘어나게 바꿈
x = GetZeroStartX(x, 'ElapsedMth')
Execute(maxDim, x, vars, tag+'_2_prodCode+ElapsedMth')

x = xBase.copy(True)
vars = ['ProdCode', 'ElapsedMth', 'AttainedAge'] 
# 경과월 --> 0에서부터 1씩 늘어나게 바꿈
x = GetZeroStartX(x, 'ElapsedMth')
# 가입나이 --> 경과나이로 대체
x['AttainedAge'] = x.apply(lambda row: int(row['ContAge'] + row['ElapsedMth']//12), axis=1)
del x['ContAge']
x = GetZeroStartX(x, 'AttainedAge')
Execute(maxDim, x, vars, tag+'_2_prodCode+ElapsedMth+AttainedAge')

start learning 1 2020-12-27 04:38... end learning 2020-12-27 04:47
start learning 2 2020-12-27 04:47... end learning 2020-12-27 04:56
start learning 3 2020-12-27 04:56... end learning 2020-12-27 05:05
start learning 4 2020-12-27 05:05... end learning 2020-12-27 05:15
start learning 5 2020-12-27 05:15... end learning 2020-12-27 05:24
emb_tf700 max5_1_prodCode ['126.08855726271979', [123.9236085115317, 129.29563314393016, 125.44786803755956, 127.2362496621945, 124.53942695838306]]
start learning 1 2020-12-27 05:27... end learning 2020-12-27 05:37
start learning 2 2020-12-27 05:37... end learning 2020-12-27 05:46
start learning 3 2020-12-27 05:46... end learning 2020-12-27 05:55
start learning 4 2020-12-27 05:55... end learning 2020-12-27 06:04
start learning 5 2020-12-27 06:04... end learning 2020-12-27 06:14
emb_tf700 max5_2_prodCode+ElapsedMth ['129.5552611847592', [130.3043684504528, 131.25047878038924, 127.66846671372359, 129.1149402096793, 129.4380517695509]]
start learning 1 2020-1

In [ ]:
maxDim = 3
tag = 'max3'

x = xBase.copy(True)
vars = ['ProdCode']
Execute(maxDim, x, vars, tag+'_1_prodCode')

x = xBase.copy(True)
vars = ['ProdCode', 'ElapsedMth']
# 경과월 --> 0에서부터 1씩 늘어나게 바꿈
x = GetZeroStartX(x, 'ElapsedMth')
Execute(maxDim, x, vars, tag+'_2_prodCode+ElapsedMth')

x = xBase.copy(True)
vars = ['ProdCode', 'ElapsedMth', 'AttainedAge'] 
# 경과월 --> 0에서부터 1씩 늘어나게 바꿈
x = GetZeroStartX(x, 'ElapsedMth')
# 가입나이 --> 경과나이로 대체
x['AttainedAge'] = x.apply(lambda row: int(row['ContAge'] + row['ElapsedMth']//12), axis=1)
del x['ContAge']
x = GetZeroStartX(x, 'AttainedAge')
Execute(maxDim, x, vars, tag+'_2_prodCode+ElapsedMth+AttainedAge')

start learning 1 2020-12-27 22:37... end learning 2020-12-27 22:47
start learning 2 2020-12-27 22:47... end learning 2020-12-27 22:57
start learning 3 2020-12-27 22:57... end learning 2020-12-27 23:07
start learning 4 2020-12-27 23:07... end learning 2020-12-27 23:17
start learning 5 2020-12-27 23:17... end learning 2020-12-27 23:27
emb_rf700 max3_1_prodCode ['125.27610332314343', [129.20802835682355, 124.86064804535026, 125.7826631418312, 124.63279553095764, 121.8963815407545]]
start learning 1 2020-12-27 23:30... end learning 2020-12-27 23:40
start learning 2 2020-12-27 23:40... end learning 2020-12-27 23:50
start learning 3 2020-12-27 23:50... end learning 2020-12-28 00:00
start learning 4 2020-12-28 00:00... end learning 2020-12-28 00:11
start learning 5 2020-12-28 00:11... end learning 2020-12-28 00:21
emb_rf700 max3_2_prodCode+ElapsedMth ['127.85011086114415', [128.95183606330804, 125.9892861237377, 131.5672993166055, 127.95143611694257, 124.79069668512699]]
start learning 1 2020

In [ ]:
########### prodCode, prodCode+Inter_EM_AA
maxDim = 3
tag = 'max3'

x = xBase.copy(True)

x['ElapsedMth_'] = x['ElapsedMth']
x = GetDigitized(x, 'ElapsedMth_', 20)

x['AttainedAge'] = x.apply(lambda row: int(row['ContAge'] + row['ElapsedMth']//12), axis=1)
del x['ContAge']
x['AttainedAge_'] = x['AttainedAge']
x = GetDigitized(x, 'AttainedAge_', 20)

x['Inter_EM_AA'] = x.apply(lambda row: str(int(row.ElapsedMth_)) + '_' + str(int(row.AttainedAge_)), axis=1)
del x['ElapsedMth_']
del x['AttainedAge_']
x = GetNumerical(x, 'Inter_EM_AA')

vars = ['ProdCode', 'Inter_EM_AA']
Execute(maxDim, x, vars, tag+'_2_prodCode+Inter_EM_AA')


In [ ]:
########### prodCode, prodCode+Inter_EM_AA
# 엔티티 임베딩 적용한 후 csv 저장

# 의미있는 변수생성을 마친 데이터
x = pd.read_csv('d:/x.csv')
y = pd.read_csv('d:/y.csv')

maxDim = 3
tag = 'max3'

x['ElapsedMth_'] = x['ElapsedMth']
x = GetDigitized(x, 'ElapsedMth_', 20)

x['AttainedAge_'] = x['AttainedAge']
x = GetDigitized(x, 'AttainedAge_', 20)

x['Inter_EM_AA'] = x.apply(lambda row: str(int(row.ElapsedMth_)) + '_' + str(int(row.AttainedAge_)), axis=1)
del x['ElapsedMth_']
del x['AttainedAge_']
x = GetNumerical(x, 'Inter_EM_AA')

vars = ['ProdCode', 'Inter_EM_AA']
x = GetXEmbApplied(maxDim, x, vars)

x.to_csv('d:/x.csv', index=False)
y.to_csv('d:/y.csv', index=False)
